# COCO sample - animal or person?

In [148]:
import os
import glob
import fiftyone as fo
from PIL import Image
import numpy as np
import json
import cv2
import matplotlib.pyplot as plt

## 1. Images load

In [183]:
input_dir = 'data'
labels_path = './instances_default.json'

input_img_paths = [i for i in os.listdir('data/') if i.endswith('.jpg')]

In [184]:
input_img_paths

['COCO_test2014_000000000970.jpg',
 'COCO_test2014_000000000568.jpg',
 'COCO_test2014_000000001024.jpg',
 'COCO_test2014_000000000448.jpg',
 'COCO_test2014_000000001793.jpg',
 'COCO_test2014_000000001380.jpg',
 'COCO_test2014_000000001551.jpg',
 'COCO_test2014_000000000840.jpg',
 'COCO_test2014_000000002945.jpg',
 'COCO_test2014_000000000202.jpg']

## 2. Annotation load

In [97]:
with open('instances_default.json', 'r') as f:
    annotations = json.load(f)

In [193]:
annotations.keys()

dict_keys(['licenses', 'info', 'categories', 'images', 'annotations'])

In [198]:
print('licenses \n', annotations['licenses'])
print('\n info \n', annotations['info'])
print('\n categories \n', annotations['categories'])
print('\n images[0] \n', annotations['images'][0])
print('\n annotations[0] \n', annotations['annotations'][0])

licenses 
 [{'name': '', 'id': 0, 'url': ''}]

 info 
 {'contributor': '', 'date_created': '', 'description': '', 'url': '', 'version': '', 'year': ''}

 categories 
 [{'id': 1, 'name': 'person', 'supercategory': ''}, {'id': 2, 'name': 'animal', 'supercategory': ''}]

 images[0] 
 {'id': 1, 'width': 425, 'height': 640, 'file_name': 'COCO_test2014_000000000202.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}

 annotations[0] 
 {'id': 1, 'image_id': 1, 'category_id': 1, 'segmentation': [], 'area': 65726.3808, 'bbox': [35.3, 89.0, 133.46, 492.48], 'iscrowd': 0, 'attributes': {'occluded': False, 'rotation': 0.0}}


In [253]:
def parsing(img_name = None):

    anno_categ = annotations['categories']
    img_id = None
    for anno_images in annotations['images']:
        if anno_images['file_name'] == img_name:
            anno_images_id = anno_images['id']

    bboxes = []
    labels = []
    
    for anno in annotations['annotations']:
        if anno['image_id'] == anno_images_id:
            bboxes.append(anno['bbox'])
            labels.append(anno_categ[anno['category_id']-1]['name'])
        
    return bboxes, labels

In [254]:
len(input_img_paths)

10

In [241]:
images = []
bboxes = []

for i in range(len(input_img_paths)):
    bboxes, labels = parsing(input_img_paths[i])

    xywh = np.array(bboxes)
    xyxy = xywh.copy()

    # coco format [left, top, width, high] --> [left, top, right, bottom]
    xyxy[:, 2] = xywh[:, 0] + (xywh[:, 2])
    xyxy[:, 3] = xywh[:, 1] + (xywh[:, 3])

    img = np.array(Image.open('data/'+input_img_paths[i]))
    xyxy
    
    for idx in range(len(xyxy)):
        img_box.append(cv2.rectangle(img, (list(map(int, xyxy[idx][:2]))), (list(map(int, xyxy[idx][2:]))), (255, 255, 0), 3))
        

In [255]:
def visualize_bbox(img, bbox, class_name, color=(255, 0, 0), thickness=5):
    x_min, y_min, x_max, y_max = map(int, bbox)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, thickness)
    
    return img

def plot_examples(images, bboxes=None, class_name=None):
    fig = plt.figure(figsize=(15, 15))
    columns = 5
    rows = 2

    for i in range(1, len(images)+1):
        img = visualize_bbox(images[i-1], bboxes[i-1], class_name=class_name[i-1])

        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
    plt.show()
    
# example 
# images = [array(), array(), ...]
# bboxes = [[x1,y1, x2,y2], [x1,y1, x2,y2], ...]
# class_name = [animal, human, ...]

29

# BBV 사용해보기

In [232]:
import bbox_visualizer as bbv

In [234]:
input_img_paths[5]

'COCO_test2014_000000001380.jpg'

In [ ]:
bbv.draw_rectangle(img, bbox)
bbv.add_label(img, label, bbox, top=True)

In [248]:
import json

import bbox_visualizer as bbv
import cv2

img = cv2.imread('./data/COCO_test2014_000000001380.jpg')
annotation = json.load(open('./instances_default.json'))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [246]:
# labels = []
# bboxes = []
# for shape in annotation['shapes']:
#     labels.append(shape['label'])
#     mins = shape['points'][0]
#     maxs = shape['points'][1]
#     bboxes.append(mins + maxs)

img_with_boxes = bbv.draw_multiple_rectangles(img, bboxes)
img_with_boxes_2 = img_with_boxes.copy()

img_with_boxes = bbv.add_multiple_labels(img_with_boxes, labels, bboxes)
img_with_T_labels = bbv.add_multiple_T_labels(img_with_boxes_2, labels, bboxes)

img_with_flags = bbv.draw_multiple_flags_with_labels(img, labels, bboxes)

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2


In [238]:
annotation = json.load(open('./instances_default.json'))
annotation

{'licenses': [{'name': '', 'id': 0, 'url': ''}],
 'info': {'contributor': '',
  'date_created': '',
  'description': '',
  'url': '',
  'version': '',
  'year': ''},
 'categories': [{'id': 1, 'name': 'person', 'supercategory': ''},
  {'id': 2, 'name': 'animal', 'supercategory': ''}],
 'images': [{'id': 1,
   'width': 425,
   'height': 640,
   'file_name': 'COCO_test2014_000000000202.jpg',
   'license': 0,
   'flickr_url': '',
   'coco_url': '',
   'date_captured': 0},
  {'id': 2,
   'width': 640,
   'height': 427,
   'file_name': 'COCO_test2014_000000000448.jpg',
   'license': 0,
   'flickr_url': '',
   'coco_url': '',
   'date_captured': 0},
  {'id': 3,
   'width': 640,
   'height': 425,
   'file_name': 'COCO_test2014_000000000568.jpg',
   'license': 0,
   'flickr_url': '',
   'coco_url': '',
   'date_captured': 0},
  {'id': 4,
   'width': 640,
   'height': 424,
   'file_name': 'COCO_test2014_000000000840.jpg',
   'license': 0,
   'flickr_url': '',
   'coco_url': '',
   'date_captured